In [3]:
import time
start = time.time()
import os
import slate3k

import numpy as np
import pandas as pd

#datetime 
from datetime import datetime



import sys
sys.path.append("../..")

# Data Inputs

In [4]:
# directory where the data are stored
data_dir = "/data/"

mydir = os.getcwd() + data_dir # This gives us the path from where the python file is executed
files = os.listdir(mydir) #these are the differents pdf files

nb_files = len(files)

# Read PDF

Here, the library let's read the files.

In [5]:
start = time.time()
#let's set up a dataframe to store each row of the excel file

#read the excel into pandas 
df = pd.read_excel('template.xlsx')

#we need to adjust the excel sheet to be adapted to the data we have in the PDF
df = df.drop('zero_eighteen', axis = 1)
df = df.drop('sixty_four', axis = 1)
df.rename(columns={'nineteen_twenty':'zero_twenty'}, inplace=True)
df.rename(columns={'sixty_five_plus':'sixty_four_plus'}, inplace=True)


In [6]:
count_ok = 0
count_not_ok = 0
for file in files:
    with open(file, 'rb') as f:
        #open each doc
        doc = slate3k.PDF(f)
        nb_pages = len(doc)
        for n in range(nb_pages):
            try:
                #here we gather the data at the columns of the excel
                meta_doc = doc[n].split('\n')
                start_date = meta_doc[0].split(':')[1].split('-')[0][1:-1]
                end_date = meta_doc[0].split(':')[1].split('-')[1][1:-1]
                product_name = meta_doc[14]
                state = meta_doc[2][:2]
                group_rating_areas = meta_doc[6][4:6]
                
                #here we gather the columns corresponding to the different ages
                list_rate = list()
                for i in range(1,4):
                    elem = doc[n].split('Age')[i].split('Rate')[1].split('\n')[1:-2]
                    list_rate.extend(elem)
                row_excel = [start_date, end_date, product_name, state, group_rating_areas] + list_rate
                df.loc[len(df)] = row_excel
                count_ok +=1
            except : 
                count_not_ok +=1

writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

 

In [7]:
end = time.time()

print(count_ok, ' sheets are done.')       
print(count_not_ok, ' sheets are not done.')

print('It took', round(end-start), 'seconds to run the code.')

288  sheets are done.
72  sheets are not done.
It took 25 seconds to run the code.
